# SQL HW


In [1]:
SQL_SOURCE = 'https://fb55.carto.com/api/v2/sql?q='

import urllib2
import urllib
import StringIO
import ast
import pandas as pd

def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return response.read()

## Task 1 — Familiarize with SQL Clauses

### 1.1 Sort data by start_station_id, tripduration 
### 1.2.1 Only show the top 10 records (aka head in SQL)
### 1.2.2 Only show the last 10 records (aka tail in SQL) 
### 1.3 List all unique start_station_id values
### 1.4 Count the number of trips (aka wc -l in SQL)
### 1.5 Find the average/min/max trip duration

In [15]:
task1_1 = '''
SELECT * FROM citibike
WHERE tripduration <= 10800
ORDER BY start_station_id
'''

task1_2_1 = '''
SELECT * FROM citibike

WHERE tripduration <= 10800

ORDER BY start_station_id

LIMIT 10 
'''

task1_2_2 = '''
SELECT * FROM citibike

WHERE tripduration <= 10800

ORDER BY start_station_id DESC

LIMIT 10 
'''

task1_3 = '''
SELECT DISTINCT start_station_id FROM citibike
'''

task1_4 = '''
SELECT COUNT(tripduration) FROM citibike
'''

task1_5 = '''
SELECT AVG(tripduration) as myavg, MAX(tripduration) as mymax, MIN(tripduration) as mymin FROM citibike
'''

In [16]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task1_1)), sep=',').head()

,cartodb_id,the_geom,the_geom_webmercator,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,75,NaN,NaN,511,2/1/2015 0:50,2/1/2015 0:59,119,Park Ave & St Edwards St,40.696089,-73.978034,420,Clermont Ave & Lafayette Ave,40.687645,-73.969689,20792,Subscriber,1982.0,1
1,166,NaN,NaN,521,2/1/2015 2:29,2/1/2015 2:38,128,MacDougal St & Prince St,40.727103,-74.002971,276,Duane St & Greenwich St,40.717488,-74.010455,15023,Subscriber,1965.0,1
2,26,NaN,NaN,907,2/1/2015 0:17,2/1/2015 0:32,128,MacDougal St & Prince St,40.727103,-74.002971,531,Forsyth St & Broome St,40.718939,-73.992663,16917,Subscriber,1993.0,1
3,92,NaN,NaN,731,2/1/2015 1:11,2/1/2015 1:23,128,MacDougal St & Prince St,40.727103,-74.002971,502,Henry St & Grand St,40.714215,-73.981346,19506,Subscriber,1970.0,1
4,142,NaN,NaN,428,2/1/2015 1:54,2/1/2015 2:02,128,MacDougal St & Prince St,40.727103,-74.002971,312,Allen St & E Houston St,40.722055,-73.989111,21462,Subscriber,1983.0,1


In [17]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task1_2_1)), sep=',')

,cartodb_id,the_geom,the_geom_webmercator,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,75,NaN,NaN,511,2/1/2015 0:50,2/1/2015 0:59,119,Park Ave & St Edwards St,40.696089,-73.978034,420,Clermont Ave & Lafayette Ave,40.687645,-73.969689,20792,Subscriber,1982,1
1,92,NaN,NaN,731,2/1/2015 1:11,2/1/2015 1:23,128,MacDougal St & Prince St,40.727103,-74.002971,502,Henry St & Grand St,40.714215,-73.981346,19506,Subscriber,1970,1
2,166,NaN,NaN,521,2/1/2015 2:29,2/1/2015 2:38,128,MacDougal St & Prince St,40.727103,-74.002971,276,Duane St & Greenwich St,40.717488,-74.010455,15023,Subscriber,1965,1
3,165,NaN,NaN,224,2/1/2015 2:28,2/1/2015 2:32,128,MacDougal St & Prince St,40.727103,-74.002971,250,Lafayette St & Jersey St,40.724561,-73.995653,15715,Subscriber,1985,1
4,26,NaN,NaN,907,2/1/2015 0:17,2/1/2015 0:32,128,MacDougal St & Prince St,40.727103,-74.002971,531,Forsyth St & Broome St,40.718939,-73.992663,16917,Subscriber,1993,1
5,142,NaN,NaN,428,2/1/2015 1:54,2/1/2015 2:02,128,MacDougal St & Prince St,40.727103,-74.002971,312,Allen St & E Houston St,40.722055,-73.989111,21462,Subscriber,1983,1
6,81,NaN,NaN,665,2/1/2015 0:53,2/1/2015 1:04,150,E 2 St & Avenue C,40.720874,-73.980858,497,E 17 St & Broadway,40.737050,-73.990093,18344,Subscriber,1984,1
7,87,NaN,NaN,671,2/1/2015 0:58,2/1/2015 1:09,151,Cleveland Pl & Spring St,40.721816,-73.997203,433,E 13 St & Avenue A,40.729554,-73.980572,14706,Subscriber,1989,1
8,106,NaN,NaN,270,2/1/2015 1:22,2/1/2015 1:27,151,Cleveland Pl & Spring St,40.721816,-73.997203,2010,Grand St & Greene St,40.721655,-74.002347,19269,Subscriber,1980,1
9,163,NaN,NaN,763,2/1/2015 2:25,2/1/2015 2:38,151,Cleveland Pl & Spring St,40.721816,-73.997203,476,E 31 St & 3 Ave,40.743943,-73.979661,21554,Subscriber,1980,1


In [18]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task1_2_2)), sep=',')

,cartodb_id,the_geom,the_geom_webmercator,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,105,NaN,NaN,1217,2/1/2015 1:22,2/1/2015 1:43,2022,E 59 St & Sutton Pl,40.758491,-73.959206,477,W 41 St & 8 Ave,40.756405,-73.990026,15676,Subscriber,1979,2
1,124,NaN,NaN,469,2/1/2015 1:37,2/1/2015 1:45,2021,W 45 St & 8 Ave,40.759291,-73.988597,480,W 53 St & 10 Ave,40.766697,-73.990617,16528,Subscriber,1975,1
2,4,NaN,NaN,818,2/1/2015 0:01,2/1/2015 0:15,2004,6 Ave & Broome St,40.724399,-74.004704,505,6 Ave & W 33 St,40.749013,-73.988484,21044,Subscriber,1985,2
3,145,NaN,NaN,999,2/1/2015 1:58,2/1/2015 2:14,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,307,Canal St & Rutgers St,40.714275,-73.989900,17365,Subscriber,1966,1
4,67,NaN,NaN,918,2/1/2015 0:44,2/1/2015 0:59,539,Metropolitan Ave & Bedford Ave,40.715348,-73.960241,372,Franklin Ave & Myrtle Ave,40.694528,-73.958089,18410,Subscriber,1989,1
5,53,NaN,NaN,1101,2/1/2015 0:34,2/1/2015 0:52,539,Metropolitan Ave & Bedford Ave,40.715348,-73.960241,261,Johnson St & Gold St,40.694749,-73.983625,14663,Subscriber,1992,1
6,178,NaN,NaN,413,2/1/2015 2:57,2/1/2015 3:03,537,Lexington Ave & E 24 St,40.740259,-73.984092,433,E 13 St & Avenue A,40.729554,-73.980572,16235,Subscriber,1988,1
7,40,NaN,NaN,285,2/1/2015 0:23,2/1/2015 0:28,531,Forsyth St & Broome St,40.718939,-73.992663,403,E 2 St & 2 Ave,40.725029,-73.990697,19261,Subscriber,1966,1
8,35,NaN,NaN,977,2/1/2015 0:23,2/1/2015 0:39,531,Forsyth St & Broome St,40.718939,-73.992663,127,Barrow St & Hudson St,40.731724,-74.006744,15706,Subscriber,1984,1
9,159,NaN,NaN,497,2/1/2015 2:21,2/1/2015 2:29,528,2 Ave & E 31 St,40.742909,-73.977061,236,St Marks Pl & 2 Ave,40.728419,-73.987140,18299,Subscriber,1982,1


In [19]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task1_3)), sep=',').head()

,start_station_id,Unnamed: 1
0,539,NaN
1,195,NaN
2,285,NaN
3,498,NaN
4,483,NaN


In [20]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task1_4)), sep=',')

,count,Unnamed: 1
0,199,NaN


In [21]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task1_5)), sep=',')

,myavg,mymax,mymin
0,775.527638,35251,89


## Task 2 — Working with date/time

### 2.1 Selecting trips started on Feb-02-2015 only

### 2.2 Selecting trips started on the weekends 

### 2.3 Average Trip Duration on Weekends

### 2.4 Average Trip Duration on Weekdays

In [25]:
task2_1 = '''
SELECT * FROM citibike

WHERE starttime >= '2015-02-02 00:00'

AND starttime < '2015-02-03 00:00'
'''

task2_2 = '''
SELECT * FROM citibike

WHERE extract(DOW FROM starttime) in (0,6)
'''

task2_3 = '''
SELECT AVG(tripduration) as weekend_avg FROM citibike

WHERE extract(DOW FROM starttime) in (0,6) 
'''

task2_4 = '''
SELECT AVG(tripduration) as weekend_avg FROM citibike

WHERE extract(DOW FROM starttime) not in (0,6) 
'''

In [24]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task2_1)), sep=',')

,cartodb_id,the_geom,the_geom_webmercator,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender


In [26]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task2_2)), sep=',')

ValueError: function pg_catalog.date_part(unknown, text) does not exist

In [27]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task2_3)), sep=',')

ValueError: function pg_catalog.date_part(unknown, text) does not exist

In [28]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task2_4)), sep=',')

ValueError: function pg_catalog.date_part(unknown, text) does not exist

## Task 3 — Working with Space
### 3.1 Showing the list of start station locations 
### 3.2 Showing the number of trips started per station
### 3.3 Showing the number of trips started per station 500 M from Times Square

In [33]:
task3_1 = '''
SELECT start_station_latitude, start_station_longitude FROM citibike

GROUP BY start_station_latitude, start_station_longitude
'''

task3_2 = '''
SELECT COUNT(tripduration) FROM citibike

GROUP BY start_station_latitude, start_station_longitude
'''

task3_3 = '''
SELECT CDB_TransformToWebmercator(CDB_LatLng(start_station_latitude, start_station_longitude)) as the_geom_webmercator, 
	MIN(cartodb_id) as cartodb_id,
    COUNT(tripduration) as numtrips

FROM citibike

WHERE ST_DWithin(
  		CDB_LatLng(start_station_latitude, start_station_longitude)::geography,
  		CDB_LatLng(40.7577,-73.9857)::geography,
  		500)
        
GROUP BY start_station_latitude, start_station_longitude
'''

In [40]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task3_1)), sep=',').head()

,start_station_latitude,start_station_longitude
0,40.764397,-73.973715
1,40.750977,-73.987654
2,40.728145,-73.990214
3,40.727103,-74.002971
4,40.729387,-73.977724


In [36]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task3_2)), sep=',').head()

,count,Unnamed: 1
0,1,NaN
1,1,NaN
2,3,NaN
3,5,NaN
4,3,NaN


In [39]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task3_3)), sep=',')

,the_geom_webmercator,cartodb_id,numtrips
0,0101000020110F000004AF971C1D6B5FC17A48F3C1A8FB...,54,1
1,0101000020110F0000F55695027D6B5FC186E5BB69D7FB...,33,3
2,0101000020110F00009BB87D02B76A5FC146C144E9E5FB...,19,1
3,0101000020110F00001607D538556B5FC119A6CA6F41FC...,124,1
4,0101000020110F000020D6742CE16A5FC112F4D73D73FC...,42,2


## Task 4 — Putting it all together
### 4.1 Find the station that had the longest average trip duration during the weekend and within 500m of TimeSquare! 

### 4.2 create lines for trips started from stations within 500m of Times Squares and lasted less than 2 hours.



In [41]:
task4_1 = '''
SELECT CDB_TransformToWebmercator(CDB_LatLng(start_station_latitude, start_station_longitude)) as the_geom_webmercator, 
	MIN(start_station_id) as station_id,
    COUNT(tripduration) as numtrips,
    AVG(tripduration) as avg_tripduration

FROM citibike

WHERE ST_DWithin(
  		CDB_LatLng(start_station_latitude, start_station_longitude)::geography,
  		CDB_LatLng(40.7577,-73.9857)::geography,
  		500)
        AND extract(DOW FROM starttime) in (0,6)
        

GROUP BY start_station_latitude, start_station_longitude

ORDER by avg_tripduration desc

LIMIT 1
'''

task4_2 = '''
SELECT ST_MakeLine(CDB_TransformToWebmercator(CDB_LatLng(start_station_latitude, start_station_longitude)),
                   CDB_TransformToWebmercator(CDB_LatLng(end_station_latitude, end_station_longitude)))
    as the_geom_webmercator,
	MIN(cartodb_id) as cartodb_id,
    COUNT(tripduration) as numtrips

FROM citibike

WHERE ST_DWithin(
  		CDB_LatLng(start_station_latitude, start_station_longitude)::geography,
  		CDB_LatLng(40.7577,-73.9857)::geography,
  		500)
        
        AND tripduration<7200
        
GROUP BY start_station_latitude, start_station_longitude, end_station_latitude, end_station_longitude
'''

In [42]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task4_1)), sep=',')

ValueError: function pg_catalog.date_part(unknown, text) does not exist

In [43]:
pd.read_csv(StringIO.StringIO(queryCartoDB(task4_2)), sep=',')

,the_geom_webmercator,cartodb_id,numtrips
0,0102000020110F00000200000004AF971C1D6B5FC17A48...,54,1
1,0102000020110F000002000000F55695027D6B5FC186E5...,179,1
2,0102000020110F000002000000F55695027D6B5FC186E5...,182,1
3,0102000020110F000002000000F55695027D6B5FC186E5...,33,1
4,0102000020110F0000020000009BB87D02B76A5FC146C1...,19,1
5,0102000020110F0000020000001607D538556B5FC119A6...,124,1
6,0102000020110F00000200000020D6742CE16A5FC112F4...,42,1
7,0102000020110F00000200000020D6742CE16A5FC112F4...,76,1
